In [1]:
from os import listdir
import os
import numpy as np
import operator

In [2]:
def MNNB(sub_dict, words, total):
    prob = -1
    for word in words:
        if(word in sub_dict):
            prob += (np.log2(sub_dict[word] + 1) - np.log2(sub_dict["total_count"] + total))
    prob += (np.log2(sub_dict["total_count"]) - np.log2(total))
    return prob

In [3]:
def find_prediction(dictionary, file_path):
    l_words = read_file(file_path)
    words = clean_data(l_words)
    prob = -1
    best_prob = -1000000
    news_grp = ''
    dict_keys = dictionary.keys()
    for key in dict_keys:
        if(key != 'total'):
            prob = MNNB(dictionary[key], words, dictionary["total"])
            print(prob)
            if(prob > best_prob):
                news_grp = key
                best_prob = prob
    return news_grp

In [4]:
def predict(dictionary, path):
    files_path = os.listdir(path)
    predictions = []
    for file_path in files_path:
        pred = find_prediction(dictionary, path + "\\" + file_path)
        predictions.append(pred)
    return predictions

In [5]:
def files_path(base_path):
    folders_name = listdir(base_path)
    folders_path = []
    for folder_name in folders_name:
        folders_path.append(base_path + "\\" + folder_name)
    file_path = []
    for folder_path in folders_path:
        l = os.listdir(folder_path)
        for i in l:
            file_path.append(folder_path + "\\" + i)
    return folders_name, folders_path, file_path

In [6]:
def clean_data(l):
    stopwords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", " ", ".", ",", ":", ";", "-", "--", "?", "/", "\\", '']
    for word in list(l):
        if(word in stopwords or not word.isalpha()):
            l.remove(word)
    return l

In [7]:
def read_file(path):
    l = []
    i = 0
    myfile = open(path, 'r')
    for line in myfile.readlines():
        if (i > 12):
            x = (line.strip().split(" "))
            for w in x:
                l.append(w.lower())
        i += 1
    return l

In [8]:
def build_word_dictionary(list_words):
    dictionary_words = {}
    for word in list_words:
        if(word not in dictionary_words.keys()):
            dictionary_words[word] = 1
        else:
            dictionary_words[word] += 1
    return dictionary_words

In [9]:
def make_final_dict(dict_imp_words, folders_name, folders_path):
    dictionary = {}
    wd_keys = dict_imp_words.keys()
    for i in range(len(folders_name)):
        feature = folders_name[i]
        dictionary["total"] = len(dict_imp_words)
        dictionary[feature] = dict_imp_words.copy()
        path = folders_path[i]
        folder_i_files = os.listdir(path)
        for folder_i_file in folder_i_files:
            file_words = read_file(path + "\\" + folder_i_file)
            pure_words = clean_data(file_words)
            for word in pure_words:
                if(word not in wd_keys):
                    continue
                else:
                    dictionary[feature][word] += 1
        dictionary[feature]["total_count"] = sum(dictionary[feature].values())
    return dictionary, wd_keys

In [10]:
def dict_build(base_path, n):
    folders_name, folders_path, file_path = files_path(base_path)
    list_words = []
    for file in file_path:
        file_words = read_file(file)
        pure_words = clean_data(file_words)
        list_words += pure_words
    dictionary_words = build_word_dictionary(list_words)
    sorted_dict = sorted(dictionary_words.items(), key=operator.itemgetter(1), reverse = True)
    imp_words = sorted_dict[0:n]
    dict_imp_words = {}
    for i in range(len(imp_words)):
        if(imp_words[i][0] not in dict_imp_words.keys()):
            x = imp_words[i][0]
            dict_imp_words[x] = 0
    dictionary, wd_keys = make_final_dict(dict_imp_words, folders_name, folders_path)
    return dictionary, wd_keys

In [11]:
# dictionary,wd_keys = dict_build("C:\\Users\\abhishek\\Desktop\\Abhi\\attachment24373a82d9ccf7a6e358b6f1c57de88edd461\\20_newsgroups_train", 5500)

# def save_obj(dictionary, name ):
#     with open('obj/'+ name + '.pkl', 'wb') as f:
#         pickle.dump(dictionary, f, pickle.HIGHEST_PROTOCOL)



In [12]:
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [13]:
p = "C:\\Users\\abhishek\\Downloads\\Compressed\\20_newsgroups_test\\alt.atheism"
pr = predict(dictionary, p)

-241.10017759775488
-292.1172174237145
-288.63463060757573
-291.01379729901174
-280.7633331531925
-287.18807845263734
-290.714773590303
-279.7240356979934
-279.77120072056357
-292.20882567726454
-291.99074918166957
-284.820117372548
-280.33503665403936
-284.66924734383724
-285.7439959485308
-262.632571090555
-266.0173277285902
-266.1505679708397
-265.60093432350783
-239.87171787330473
-614.6122033661024
-747.870690711765
-729.0045620868726
-740.4576348630285
-734.6622079412572
-760.1482311132442
-745.0324623204008
-713.7622058580348
-723.3108089081088
-711.7540926868425
-750.1815234380653
-752.5753044907907
-745.9375690055963
-716.3597549588733
-701.7034539879883
-714.7269535664674
-714.1441304124672
-722.9520069711795
-708.2010797833806
-610.4059068110473
-533.4658320431549
-611.6103310265561
-606.336401728825
-599.7843793479061
-602.4202681029036
-596.2440317750505
-629.77676507543
-574.2207180503408
-580.5186865257452
-593.631226222241
-598.5268537610596
-592.2331488006905
-596.0053

-1062.1970594153001
-1087.6673913781706
-1032.6276857131056
-994.1789138674694
-839.1056521827438
-923.8595167019045
-918.8530722839879
-934.7301644113649
-913.4415331731814
-940.801058920293
-932.1951485365773
-909.5450841390331
-902.6433901543645
-906.8442320760333
-918.3022344262754
-901.9578031829154
-903.6651617124168
-878.9423216034182
-904.9874823573862
-870.9400287693925
-887.0971614271817
-899.4424823285499
-881.9238438990884
-867.2442946726533
-3154.680547872391
-3511.3434983975267
-3522.8528428639984
-3559.8939942311054
-3478.7344442312838
-3599.919098482861
-3515.0608247288906
-3432.8266169932253
-3442.221320089749
-3427.514859376288
-3476.111282607575
-3426.7363072109583
-3484.3163153182645
-3380.026519588693
-3397.239651666536
-3229.1269328556473
-3262.74727384752
-3114.726983491248
-3189.2103696348795
-3205.383731254899
-1019.920231368143
-1158.9670531282852
-1148.3034342130575
-1155.6429827890968
-1144.0868379902824
-1180.4681979059708
-1170.902969668648
-1132.046750949

-518.0740836745914
-643.9488427719324
-637.9884897982004
-646.1323859337157
-648.3477101110965
-663.2295848539569
-642.8353422138982
-614.9831543894886
-620.2041342990068
-643.0106197106535
-665.0697649937711
-614.433470486846
-644.7464555413267
-618.5157600857547
-639.007830852446
-542.2061218082027
-560.6866159560069
-570.4313302565199
-558.3025574376527
-535.1317604321428
-353.15488842325357
-396.7694585345691
-391.63827973710085
-399.9083439329144
-396.7427522866392
-395.94035724928204
-397.39463606008945
-393.4366449344043
-394.5012872480713
-407.159044014909
-411.49744487263234
-393.097841405582
-391.48729068722173
-381.5126461617341
-376.7685963977529
-359.5177212623467
-373.2050477295438
-375.91025955262745
-368.77018850088007
-357.64163409673137
-389.296171469554
-453.94240553024645
-435.2405939612473
-442.6987255599137
-428.58384933501907
-437.16419220632366
-446.4087710020521
-414.0201597888256
-428.58818601067287
-425.71495789614585
-437.86829279637993
-421.9407667354441
-4

-414.57712707713773
-428.1510604920759
-419.36525068546587
-421.4144969246134
-418.98742044144797
-440.5726602403865
-428.1366105281892
-395.3605674675837
-396.8326083185628
-403.35191506942294
-413.7206992930012
-428.8323288126643
-405.0566738087406
-414.1010137636351
-415.3225099740297
-412.41924033609314
-410.34335419125335
-412.36587533115795
-408.7951216867887
-409.5657978125569
-274.96835631087197
-316.92168034015117
-307.38088876895785
-312.2343452161912
-312.5847640992741
-321.51026064047517
-324.56813197283947
-302.5179240634273
-306.1424057200834
-296.0607084204129
-308.52995779614156
-306.43112202527414
-310.71397567302046
-306.4622362594109
-307.4722037092953
-278.24341015567984
-291.79637079692975
-292.162602466833
-292.4566530911139
-278.429927486963
-318.09711586193464
-360.11314101013795
-359.39052833294437
-365.21812814599286
-366.63423507417554
-371.4436595311343
-373.71802673597443
-358.47656185502797
-355.0220098759097
-355.5565300319908
-365.4215336448788
-348.8591

-2324.871633400685
-2329.029509045918
-2193.989562467852
-2350.7330393919656
-2338.0686189980834
-2344.2610612292547
-2164.118738498903
-435.0187597769683
-466.81424900440277
-465.1856472714663
-469.4643138611177
-471.96527165812205
-466.55037393836255
-483.9271108756102
-457.9214350496877
-446.95351499502885
-459.39314353661115
-470.83196411279465
-466.77933665441856
-462.3875417138331
-436.1824948007192
-457.0963348498847
-434.4914647432174
-454.52844192448714
-445.4222720285891
-455.31657411049804
-429.44661395953653
-82.23066481171617
-88.6516556385433
-87.19862485639791
-87.73744788452751
-93.74985304299075
-92.62427500341262
-83.65715385578882
-87.7436114559222
-81.07802469722932
-85.03402249343458
-84.22404578650394
-88.65713446355673
-85.68900813095019
-80.07998646284176
-83.44111012283506
-88.2147397711778
-82.65929264558584
-86.4347328523098
-84.60322532255122
-83.25774984208385
-330.6948249166004
-385.4154252075393
-379.7251420273029
-375.1674827021888
-373.7220739050859
-38

-265.8491759256635
-285.15423102676715
-282.6214761704372
-275.89300533117347
-278.32621539756076
-296.41257685964905
-285.2941298016871
-266.71824844829854
-270.31780455597817
-266.07271702401886
-278.4633778188013
-283.7883321244948
-277.059061394938
-273.8896419405381
-276.87957297247556
-268.2701903637002
-280.47771578034985
-284.36717557851193
-276.59432512065115
-268.0710103816844
-333.53000373312307
-377.9313509016776
-375.1331469404028
-382.95462226201124
-376.5766031312002
-389.69296832935
-378.73465845616266
-374.33377818543283
-372.19834810176405
-380.5581609162261
-382.8201651781601
-390.3379990594009
-373.51414046005095
-371.9389804442008
-369.7045381158563
-359.7111629733069
-375.26410826441133
-375.03205787060836
-367.3746543641215
-334.0653208003216
-94.71850020276773
-103.81044662887554
-98.65392617270678
-103.49710333884373
-101.221457276988
-105.6104362603335
-100.48005007340636
-98.23978684574834
-96.36870953463892
-96.67407908294479
-96.20922492646422
-100.90333318

-574.5977874115775
-585.529662590437
-599.1950224995843
-579.4951019204544
-581.6297265861003
-570.9292129339822
-591.0572866957921
-549.1462734436201
-562.649307527243
-573.578453211735
-563.9401998350556
-548.588581696927
-64.81824258519902
-73.53178649645203
-73.89611298162379
-74.07733076362821
-74.27444609509679
-73.84658607214223
-67.63437697636472
-71.8520531914169
-73.2077187824359
-70.50535932483407
-75.57128601128389
-76.94130885748099
-70.13804667477704
-68.02338291884669
-72.62244523405514
-64.62262426556323
-70.65392898342249
-69.55258978615953
-70.96856372159873
-67.25672952305459
-561.5696602708512
-612.8761319111721
-620.6922809975483
-619.035516934385
-615.9171540770707
-642.9322851944328
-619.5856065745601
-617.2492790405003
-609.6068553991564
-610.1566926426866
-620.9780325914624
-606.4926754330569
-620.5181062795524
-606.8783334538995
-603.1710705859227
-570.3171645702137
-568.8015506312651
-600.2743059112578
-572.5253409421105
-560.0035483317675
-87.62405430725747


-862.7606115582062
-846.2948972902805
-838.3416952570908
-868.189688895684
-787.3602554949355
-831.3891044542555
-830.007429116146
-816.6737517660465
-794.3667363918
-422.4737410161894
-451.3087681023766
-450.3990022346465
-451.1558220977808
-445.3501105217128
-457.014441449262
-458.96886014029786
-449.068762829051
-441.79824517627026
-451.5324163379306
-458.7755195205774
-449.1786505682856
-447.29985078136474
-452.5800145526608
-453.6519420587643
-444.4044035854191
-444.7979930151674
-438.2670469004544
-445.3986977449302
-417.9437692370485
-592.7428684893397
-678.399384030739
-656.1549504061722
-654.1184877016049
-651.1630553465313
-655.9932887190317
-683.1796411765773
-658.9069590382371
-650.2602765356039
-654.696458026734
-665.1301438345259
-647.5998796572658
-654.3116451744604
-652.9516405624829
-667.0822163140169
-589.9472468048098
-623.7133926082604
-624.5682196579389
-620.8156081861675
-581.5730869002589
-736.9045001945117
-775.3374019914787
-783.3979747006912
-794.9447753630426

In [14]:
# find how many files have been predicted right out of 200
total = 0
count = 0
# all the different groups of news
news_g = ["alt.atheism","comp.graphics","comp.os.ms-windows.misc","comp.sys.ibm.pc.hardware","comp.sys.mac.hardware","comp.windows.x","misc.forsale","rec.autos","rec.motorcycles","rec.sport.baseball","rec.sport.hockey","sci.crypt","sci.electronics","sci.med","sci.space","soc.religion.christian","talk.politics.guns","talk.politics.mideast","talk.politics.misc","talk.religion.misc"]
# number of files predicted correctly out of 200 from each group
for i in range(len(news_g)):
    for p in pr:
        if(p == news_g[i]):
            total += 1
    print(total)
    count += total
print(count)

115
117
118
119
119
119
119
122
124
125
125
125
126
131
133
150
154
160
161
200
2662


In [19]:
len(news_g)
base_path = 'C:\\Users\\abhishek\\Desktop\\Abhi\\attachment24373a82d9ccf7a6e358b6f1c57de88edd461\\20_newsgroups_train'

In [28]:
# build 2D array of word, features and count
X = []
folder_names = news_g
for i in folder_names:
    subd = dictionary[i]
    l = []
    for j in wd_keys:
        l.append(subd[j])
    X.append(l)
X = np.array(X)
X

array([[3256,  474,  414, ...,    0,    0,    0],
       [1958,  250,  289, ...,    0,    0,    0],
       [2651,  261,  295, ...,    0,    0,    0],
       ...,
       [3803,  575,  567, ...,    0,    0,    0],
       [3131,  601,  483, ...,    0,    0,    0],
       [3160,  476,  381, ...,    0,    0,    0]])

In [29]:
from sklearn.naive_bayes import MultinomialNB

In [30]:
clf = MultinomialNB()
clf.fit(X, folder_name)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
clf.predict(X)

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype='<U24')